# Análise de Segmentação de Clientes

## Objetivo

Este notebook demonstra uma análise completa de segmentação de clientes utilizando a técnica RFM (Recency, Frequency, Monetary).
A análise RFM permite identificar diferentes grupos de clientes baseados em:
- **Recency (R)**: Quão recentemente o cliente fez uma compra
- **Frequency (F)**: Com que frequência o cliente compra
- **Monetary (M)**: Quanto dinheiro o cliente gasta

Com essa segmentação, podemos criar estratégias de marketing direcionadas para cada grupo de clientes.

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configurações
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Leitura dos dados de clientes
df = pd.read_csv('../data/customer_data.csv')

# Visualizar as primeiras linhas
print('Dimensões dos dados:', df.shape)
print('\nPrimeiras 5 linhas:')
df.head()

In [ ]:
# Análise RFM (Recency, Frequency, Monetary)

# Convertendo a coluna de data para datetime
df['purchase_date'] = pd.to_datetime(df['purchase_date'])

# Data de referência (assumindo que é a data mais recente nos dados)
reference_date = df['purchase_date'].max()

# Calculando métricas RFM por cliente
rfm = df.groupby('customer_id').agg({
    'purchase_date': lambda x: (reference_date - x.max()).days,  # Recency
    'transaction_id': 'count',                                   # Frequency
    'purchase_amount': 'sum'                                     # Monetary
}).reset_index()

# Renomeando as colunas
rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

# Criando scores RFM (1-5, onde 5 é o melhor)
rfm['R_score'] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])
rfm['F_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])

# Score RFM combinado
rfm['RFM_score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

# Segmentação baseada nos scores
def segment_customers(df):
    if df['RFM_score'] in ['555', '554', '544', '545', '454', '455', '445']:
        return 'Champions'
    elif df['RFM_score'] in ['543', '444', '435', '355', '354', '345', '344', '335']:
        return 'Loyal Customers'
    elif df['RFM_score'] in ['512', '511', '422', '421', '412', '411', '311']:
        return 'Potential Loyalists'
    elif df['RFM_score'] in ['533', '532', '531', '523', '522', '521', '515', '514', '513', '425', '424', '413', '414', '415', '315', '314', '313']:
        return 'New Customers'
    elif df['RFM_score'] in ['155', '154', '144', '214', '215', '115', '114']:
        return 'At Risk'
    elif df['RFM_score'] in ['155', '154', '144', '214', '215', '115']:
        return "Can't Lose Them"
    else:
        return 'Others'

rfm['segment'] = rfm.apply(segment_customers, axis=1)

print('Análise RFM concluída!')
print('\nPrimeiras linhas da análise RFM:')
rfm.head()

In [ ]:
# Gráfico de barras dos segmentos de clientes

# Contagem de clientes por segmento
segment_counts = rfm['segment'].value_counts()

# Criando o gráfico
plt.figure(figsize=(12, 8))

# Gráfico de barras
bars = plt.bar(segment_counts.index, segment_counts.values, 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3'])

# Personalizando o gráfico
plt.title('Distribuição de Clientes por Segmento RFM', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Segmento de Clientes', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xticks(rotation=45, ha='right')

# Adicionando valores nas barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{int(height)}', ha='center', va='bottom', fontweight='bold')

# Ajustando layout
plt.tight_layout()
plt.grid(axis='y', alpha=0.3)
plt.show()

# Estatísticas dos segmentos
print('\nEstatísticas por Segmento:')
segment_stats = rfm.groupby('segment').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean'
}).round(2)

segment_stats

## Interpretação dos Resultados

### Análise dos Segmentos de Clientes

Com base na análise RFM realizada, podemos interpretar os diferentes segmentos de clientes:

#### 1. **Champions** 🏆
- **Características**: Compraram recentemente, compram frequentemente e gastam mais
- **Estratégia**: Recompensar, podem se tornar advogados da marca, oferecer produtos novos

#### 2. **Loyal Customers** 💎
- **Características**: Gastam bem e compram regularmente
- **Estratégia**: Fidelizar ainda mais, programas de upselling, aumentar frequência de compra

#### 3. **Potential Loyalists** 🌱
- **Características**: Clientes recentes com potencial de crescimento
- **Estratégia**: Oferecer membership/programas de fidelidade, engajar com conteúdo

#### 4. **New Customers** 🆕
- **Características**: Compraram recentemente mas com baixa frequência
- **Estratégia**: Foco em onboarding, educação sobre produtos, oferecer suporte

#### 5. **At Risk** ⚠️
- **Características**: Gastavam bem mas não compram há um tempo
- **Estratégia**: Campanhas de reativação, ofertas especiais, pesquisa de satisfação

#### 6. **Can't Lose Them** 🚨
- **Características**: Eram bons clientes mas estão inativos
- **Estratégia**: Campanhas agressivas de recuperação, ofertas personalizadas

### Recomendações Estratégicas

1. **Foco nos Champions**: Investir em programas VIP e referral
2. **Desenvolver Potential Loyalists**: Criar jornadas de engajamento
3. **Reativar clientes At Risk**: Campanhas de win-back personalizadas
4. **Monitoramento contínuo**: Reavaliar segmentação mensalmente

Esta análise deve ser utilizada como base para definição de estratégias de CRM e campanhas de marketing direcionadas.